# Implementation of LSTM
## Variables to Consider
<ul>
    <li>Hidden Parameters (more seems better until 10)</li>
    <li>Hidden Layers (more than 1 seems useless, makes sense because input is few dimensions)</li>
    <li>Learning Rate (smaller learning rate requires more epoches to achieve similar results)</li>
    <li>Mini-Batch Size (seems to have slight improvement)</li>
    <li>Number Epochs</li>
    <li>Sequence Length (4)</li>
    <li>Length of Prediction (1)</li>
</ul>

In [1]:
#!pip uninstall statsmodels
#!pip install pmdarima
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from os import listdir
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("using cuda")

using cuda


In [20]:
class Parameters():
    def __init__(self, num_layers = 1, hidden_size = 10, batch_size = 100, seq_len = 6, pred_len = 3, num_epochs = 20, learning_rate = 0.01, dropout = 1):
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.dropout = dropout
    
    def __str__(self):
        return "{}-{}-{}-{}-{}-{}-{}-{}".format(self.num_layers, self.hidden_size, self.batch_size, self.seq_len, self.pred_len, self.num_epochs, self.learning_rate, self.dropout)


Following Code Adapted from ([github sample](https://github.com/spdin/time-series-prediction-lstm-pytorch/blob/master/Time_Series_Prediction_with_LSTM_Using_PyTorch.ipynb))

In [21]:
def readData(file='samples/WY.csv', freq=20):
    data = pd.read_csv(file, delimiter=',', index_col=0, parse_dates=True)

    #plt.figure(figsize=(20, 4))
    #plt.plot(data)
    #print(data)
    data = data.resample(str(freq) + 'T').mean()
    raw_values = np.asarray(data['CpuUtilizationAverage']).reshape(data.shape[0], 1)
    #plt.plot(data)
    #plt.show()
    #print(raw_values)
    return raw_values

training_set = readData()
print(training_set.shape)

(1080, 1)


In [52]:
from torch.utils.data import TensorDataset, DataLoader

# just used to store some data variables
class Data:
    pass

def large_sliding_windows(data, seq_length, pred_length=2):
    x = []
    y = []

    for i in range(len(data)-seq_length-pred_length+1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length:i+seq_length+pred_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

def get_concurrent():
    data = np.zeros((50, 21600))
    for i, state in enumerate(listdir("samples/")):
        training_set = readData("samples/"+state)
        training_set = training_set.flatten()
        data[i] = training_set
    x = []
    y = []
    
    for i in range(21600-1):
        _x = data[:,i:(i+1)]
        _y = data[:,i+1]
        x.append(_x)
        y.append(_y)
    return np.array(x), np.array(y)

def get_concurrent_features(seq_len, pred_len, state_name):
    data = np.zeros((1080, 50)) #fix hard coding
    state_id=0
    for i, state in enumerate(listdir("samples/")):
        if state_name==state:
            state_id=i
            print("using state {} with id {}".format(state, i))
        training_set = readData("samples/"+state)
        training_set = training_set.flatten()
        for j in range(data.shape[0]):
            data[j][i] = training_set[j]
    x, y = large_sliding_windows(data, seq_len, pred_len)
    y = y[:,:,state_id]
    print(y.shape)
    return x, y
    

def generateData(state_name, d, params):
    d.sc = MinMaxScaler()
    training_data = d.sc.fit_transform(training_set) # normalizes the data

    seq_length = params.seq_len # parameter
    pred_len = params.pred_len # parameter
    x, y = get_concurrent_features(seq_length, pred_len, state_name)

    batch_size= params.batch_size
    d.max_size = int(batch_size*(len(y)//batch_size))
    d.train_size = int(batch_size*(d.max_size*(0.8)//batch_size)) #train_size = int(len(y) * 0.67)
    d.test_size = d.max_size-d.train_size
    #test_size = len(y) - train_size

    d.dataX = Variable(torch.Tensor(np.array(x[0:d.max_size])))
    d.dataY = Variable(torch.Tensor(np.array(y[0:d.max_size])))

    d.trainX = Variable(torch.Tensor(np.array(x[0:d.train_size]))).to(device)
    d.trainY = Variable(torch.Tensor(np.array(y[0:d.train_size]))).to(device)
    train_data = TensorDataset(d.trainX, d.trainY)
    d.train_loader = DataLoader(train_data, shuffle=False, batch_size=batch_size) #primary

    d.testX = Variable(torch.Tensor(np.array(x[d.train_size:d.max_size]))).to(device)
    d.testY = Variable(torch.Tensor(np.array(y[d.train_size:d.max_size]))).to(device)
    #test_data = TensorDataset(testX, testY)
    test_data = TensorDataset(d.dataX, d.dataY)
    d.test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size) #primary
    return d
data = Data()
params = Parameters()
generateData(training_set, data, params)
print(data.trainX.shape)
print(data.trainY.shape)



<ipython-input-52-d8d91b4faf66>:51: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if state_name==state:


(1072, 3)
torch.Size([800, 6, 50])
torch.Size([800, 3])


In [14]:
## Generate Naive Examples

def generateNaive(d, params):
    mean = d.trainY.mean()
    mean_forecasts = np.full(d.testY.shape, mean.item())
    naive_forecasts = np.full(d.testY.shape, d.trainX[-1][-1].item())  # always predicts last provided
    four_naive_forecasts = np.ones(d.testY.shape)
    for row in four_naive_forecasts:
        for i in range(params.pred_len):
            row[i] = d.testX[i][-1].item()
    four_mean_forecasts = np.ones(d.testY.shape)
    for row, test_row in zip(four_mean_forecasts, d.testX):
        for i in range(params.pred_len):
            row[i] = np.mean(test_row.numpy())

    #naive_forecasts = naive_forecasts.reshape(-1, 1)
    #mean_forecasts = mean_forecasts.reshape(-1, 1)

    denormalize=False

    if denormalize:
        naive_forecasts = sc.inverse_transform(naive_forecasts)
        mean_forecasts = sc.inverse_transform(mean_forecasts)
        four_naive_forecasts = sc.inverse_transform(four_naive_forecasts)
        four_mean_forecasts = sc.inverse_transform(four_mean_forecasts)


    global_naive = naive_forecasts.reshape(naive_forecasts.shape[:-2] + (-1,))
    global_mean = mean_forecasts.reshape(mean_forecasts.shape[:-2] + (-1,))
    local_naive = four_naive_forecasts.reshape(four_naive_forecasts.shape[:-2] + (-1,))
    local_mean = four_mean_forecasts.reshape(four_mean_forecasts.shape[:-2] + (-1,))
    return (global_naive, global_mean, local_naive, local_mean)

naives  = generateNaive(data, params)
print(data.testX.shape)
print(data.testY.shape)


IndexError: index -1 is out of bounds for dimension 0 with size 0

In [29]:
class LSTM(nn.Module):
    def __init__(self, output_size, input_size, hidden_dim, n_layers, drop_prob=0):
        super(LSTM, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        #x = x.long()
        lstm_out, hidden = self.lstm(x, hidden)
        #print(lstm_out.shape)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        #out = self.dropout(lstm_out)
        #print(lstm_out.shape)
        #out = self.fc(lstm_out) # linear transform from hidden dims to output_size
        #print(out.shape)
        #out = out.view(batch_size, -1) #-1 means size will be infered
        #print(out.shape)
        #out = out[:,-2]
        #print(out.shape)
        #print(hidden[0][-1].shape)
        h_out = hidden[0][-1].view(-1, self.hidden_dim)
        out = self.fc(h_out)
        #print(out.shape)
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

Training

In [41]:
class Model:
    def __init__(self, lstm, h, criterion):
        self.lstm, self.h, self.criterion = lstm, h, criterion

def train(d, params):
    num_epochs = params.num_epochs#30 # seems to stabilize here, higher and it seems to overfit
    learning_rate = params.learning_rate#0.01

    input_size = 50 # required for this time series (value)
    hidden_size = params.hidden_size#2 # hyper parameter
    num_layers = params.num_layers#1 # hyper parameter

    lstm = LSTM(params.pred_len, input_size, hidden_size, num_layers, params.dropout)
    lstm.to(device)

    criterion = torch.nn.MSELoss()    # mean-squared error for regression
    #criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
    #optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

    h = lstm.init_hidden(params.batch_size)

    # Train the model
    for epoch in range(num_epochs):
        for x, y in d.train_loader:
            #x = x.to(device)
            #y = y.to(device)
            h = tuple([e.data for e in h])
            outputs, h = lstm(x, h)
            optimizer.zero_grad() # remove stored gradient

            # obtain the loss function
            #print(outputs.shape)
            #print(outputs.shape)
            #print(y.shape)
            loss = criterion(outputs, y) #.squeeze()
            #print(outputs)

            loss.backward()

            optimizer.step()
        if epoch % 10 == 0:
            print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
            #pass
            
    model = Model(lstm, h, criterion)
    return model
            
model = train(data, params)

/home/rayan/.local/lib/python3.8/site-packages/torch/nn/modules/rnn.py:47: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch: 0, loss: 151.96112
Epoch: 10, loss: 44.12098


Testing

In [31]:
def test(model, d, params, show_plot=False):
    model.lstm.eval()
    test_predict = np.zeros((data.max_size, params.pred_len))
    test_losses = []

    for i, (x, y) in enumerate(d.test_loader):
        model.h = tuple([each.data for each in model.h])
        x, y = x.to(device), y.to(device)
        output, model.h = model.lstm(x, model.h)
        test_loss = model.criterion(output.squeeze(), y)
        test_losses.append(test_loss.item())
        for j in range(params.batch_size):
            #test_predict.append(output[j].item())
            #test_predict[i*100+j] = output[j].item()
            for k in range(params.pred_len):
                test_predict[j+i*100][k] = output[j][k].item()


    #print(test_predict)

    #data_predict = test_predict.reshape(-1, 1)#test_predict.data.numpy()
    data_predict = test_predict
    dataY_plot = d.dataY.data.numpy()
    
    data_predict_transform = d.sc.inverse_transform(data_predict)
    dataY_plot_transform = d.sc.inverse_transform(dataY_plot)
    
    if show_plot:
        plt.figure(figsize=(20, 4))
        plt.axvline(x=d.train_size, c='r', linestyle='--') # data shift from train to test
        plt.plot(dataY_plot_transform)
        plt.plot(data_predict_transform)
        plt.suptitle('Time-Series Prediction')
        plt.show()

    #print(data_predict_transform)
    #print(dataY_plot_transform)
    return (data_predict[d.train_size:], dataY_plot[d.train_size:])

preds, labels = test(model, data, params)


Image is deceptive. Try ~100 samples to see a closer fit

Accuracy Calculations

In [32]:
eps = 1e-5
# Mean Average Percent Error
def mape(preds, labels, prnt=False):
    preds = preds.flatten()
    labels = labels.flatten()
    err = 0
    for i, (pred, label) in enumerate(zip(preds, labels)):
        denum = np.absolute(label) if np.round(label) !=0 else 50#np.max(labels) # this might be wrong
        err += (np.absolute(pred-label) / denum)
            
    err /= preds.shape[0]
    if prnt: print("MAPE - {}".format(err))
    return err
    
# Brier Score or Mean Squared Error
def mse(preds, labels, prnt=False):
    preds = preds.flatten()
    labels = labels.flatten()
    err = np.sum(np.power(preds-labels, 2)) / preds.shape[0]
    if prnt: print("MSE - {}".format(err))
    return err
    
def mae(preds, labels, prnt=False):
    preds = preds.flatten()
    labels = labels.flatten()
    err = (np.sum(np.absolute(preds-labels))) / preds.shape[0]
    if prnt: print("MAE - {}".format(err))
    return err
    
# Root Mean Squared Error
def rmse(preds, labels, prnt=False):
    err = np.power(mse(preds, labels), 0.5)
    if prnt: print("RMSE - {}".format(err))
    return err

# Symmetric Mean Absolute Percentage Error
# some issues in bias, but commonly used
def smape(preds, labels, prnt=False):
    preds = preds.flatten()
    labels = labels.flatten()
    err = 0
    for (pred, label) in zip(preds, labels):
        denum = np.absolute(pred)+np.absolute(label) if np.absolute(pred)+np.absolute(label) !=0 else np.max(labels) #check!!
        err += (np.absolute(pred-label) / denum)
    err /= preds.shape[0] # in textbook, also multiply by 200 but this might be for percentage?
    if prnt: print("SMAPE - {}".format(err))
    return err

#errors = [mape, mse, rmse, smape]
errors = [mae, rmse, mape]

def print_errors(preds, labels, prnt=False):
    err_results = []
    for error in errors:
        err_results.append(error(preds, labels, prnt))
    return err_results
        
print_errors(preds, labels)

for naive in naives:

    print_errors(naive, labels)

ValueError: operands could not be broadcast together with shapes (215000,) (600,) 

In [53]:
params = Parameters()
params.num_epochs=50
params.num_layers=2
params.dropout=0.5
Parameters(2, 20, 100, 6, 3, 50, 0.01, 0)

columns = ['MAE', 'RMSE', 'MAPE', 'MEAN_MAE', 'MEAN_RMSE', 'MEAN_MAPE', 'NAIVE_MAE', 'NAIVE_RMSE', 'NAIVE_MAPE']
df = pd.DataFrame(columns = columns, dtype=np.float64)
    
for i, state in enumerate(listdir("samples/")):
    training_set = readData("samples/"+state, freq=1)
    data = Data()
    generateData(state, data, params)
    #naives  = generateNaive(data, params)
    model = train(data, params)
    preds, labels = test(model, data, params)
    #print("----- {} Results -----".format(state.replace(".csv", "")))
    err_results = print_errors(preds, labels)
    #print(a_row.values)

    #for naive in naives:
    #    err_results.extend(print_errors(naive, labels))   
    err_results.extend([0, 0, 0, 0, 0, 0])
    
    row_df = pd.DataFrame([err_results], index = [state.replace(".csv", "")], columns = columns, dtype=np.float64)
    df = pd.concat([row_df, df])
    print(i)
    
    
print(df.shape)
df.to_csv("lstm-out/concur-{}.csv".format(params))


using state ND.csv with id 0
(1072, 3)
Epoch: 0, loss: 150.68330
Epoch: 10, loss: 13.03061
Epoch: 20, loss: 2.67542
Epoch: 30, loss: 2.42016
Epoch: 40, loss: 2.41325
0
using state KY.csv with id 1
(1072, 3)
Epoch: 0, loss: 38.53774
Epoch: 10, loss: 0.98634
Epoch: 20, loss: 0.90574
Epoch: 30, loss: 0.90399
Epoch: 40, loss: 0.90436
1
using state RI.csv with id 2
(1072, 3)
Epoch: 0, loss: 122.97905
Epoch: 10, loss: 47.82233
Epoch: 20, loss: 46.19347
Epoch: 30, loss: 46.17688
Epoch: 40, loss: 36.93348
2
using state CA.csv with id 3
(1072, 3)
Epoch: 0, loss: 313.55908
Epoch: 10, loss: 147.62901
Epoch: 20, loss: 124.51994
Epoch: 30, loss: 121.54853
Epoch: 40, loss: 121.18435
3
using state DE.csv with id 4
(1072, 3)
Epoch: 0, loss: 460.85693
Epoch: 10, loss: 222.34993
Epoch: 20, loss: 165.13432
Epoch: 30, loss: 152.42097
Epoch: 40, loss: 149.85263
4
using state OK.csv with id 5
(1072, 3)
Epoch: 0, loss: 215.63779
Epoch: 10, loss: 142.03969
Epoch: 20, loss: 134.06017
Epoch: 30, loss: 96.63786


Epoch: 40, loss: 331.77246
46
using state UT.csv with id 47
(1072, 3)
Epoch: 0, loss: 2415.67578
Epoch: 10, loss: 1702.83655
Epoch: 20, loss: 1299.95361
Epoch: 30, loss: 1056.56775
Epoch: 40, loss: 913.46002
47
using state LA.csv with id 48
(1072, 3)
Epoch: 0, loss: 117.05021
Epoch: 10, loss: 50.56598
Epoch: 20, loss: 42.95673
Epoch: 30, loss: 28.42779
Epoch: 40, loss: 26.35994
48
using state SC.csv with id 49
(1072, 3)
Epoch: 0, loss: 840.85248
Epoch: 10, loss: 560.76440
Epoch: 20, loss: 481.92490
Epoch: 30, loss: 460.05231
Epoch: 40, loss: 454.32501
49
(50, 9)
